In [ ]:
import pathlib
import sys

from matplotlib import pyplot as plt

sys.path.append(str(pathlib.PurePath('..', '..', 'src')))
from linear_geodesic_optimization.data import input_network

In [ ]:
path_probes = pathlib.PurePath('probes.csv')
path_links = pathlib.PurePath('links', '1742792400000.csv')

def get_curvatures(latency_threshold):
    graph = input_network.get_graph_from_paths(
        path_probes, path_links,
        epsilon=latency_threshold,
        clustering_distance=500000
    )
    curvatures = {
        (min(u, v), max(u, v)): data['ricciCurvature']
        for u, v, data in graph.edges(data=True)
    }
    return curvatures

In [ ]:
for threshold in range(1, 17):
    fig, ax = plt.subplots(1, 1)
    curvatures = list(get_curvatures(threshold).values())
    ax.hist(curvatures, range=(-2, 1))
    ax.set_title(f'$\\epsilon = {threshold}$')
    ax.set_xlabel('Ricci Curvature')
    ax.set_ylabel('Occurrence Count')
    negative_count = sum(curvature < 0. for curvature in curvatures)
    print(f'At epsilon = {threshold}, there are {negative_count} negative curvatures, with a min of {min(curvatures):0.4f}')
    plt.show()

In [ ]:
negative_edges = {}
for threshold in range(1, 17):
    curvatures = get_curvatures(threshold)
    for (u, v), curvature in curvatures.items():
        if (u, v) not in negative_edges:
            negative_edges[(u, v)] = threshold
negative_edges_sorted = list(sorted([
    (threshold, (u, v))
    for (u, v), threshold in negative_edges.items()
]))
for threshold, (u, v) in negative_edges_sorted:
    print(f'Link {u} -> {v} achieves negative curvature first at threshold {threshold}')

In [ ]:
added_edges_counts = {}
for threshold in negative_edges.values():
    if threshold not in added_edges_counts:
        added_edges_counts[threshold] = 1
    else:
        added_edges_counts[threshold] += 1
for threshold, count in sorted(added_edges_counts.items()):
    print(f'{count} new negative curvature links at threshold {threshold}')

In [ ]:
print(f'{len(negative_edges)} total links have negative curvature at some point')